In [ ]:
from frontend import App

app = App("trapped").clear()

V, F, T = app.mesh.preset("armadillo").decimate(30000).tetrahedralize().normalize()
app.asset.add.tet("armadillo", V, F, T)

scene = app.scene.create("sphere-trap")

scene.add_invisible_sphere([0,0,0],0.7).invert().radius(0.15,3).radius(100,4)
scene.add("armadillo").rotate(180,"y")

fixed = scene.build().report()
fixed.preview();

In [ ]:
param = app.session.param()
param.set("gravity", 0.0)
param.set("dt", 0.01)

session = app.session.create("armadillo").init(fixed)
session.start(param).preview();
session.stream();